In [1]:
#import requests
import os.path
import pandas as pd
import urllib.parse
import json
from io import StringIO
import numpy as np

In [2]:
repos = pd.read_csv('repos_name.csv')
repos.head()

,repo_name
0,FreeCodeCamp/FreeCodeCamp
1,firehol/netdata
2,joshbuchea/HEAD
3,braydie/HowToBeAProgrammer
4,sindresorhus/awesome


### Downloading Issues from Github

In [3]:
count = 0
skip=0
issues = []
for repo in repos.repo_name.values:
    url = 'https://api.github.com/repos/' + repo +'/issues?state=all'
    r = requests.get(url, allow_redirects=True, auth = ('asad14789','7c9971ae53237d2b10c0609772f43c9ec0e6fc1d'))
    if r.status_code==200 : 
        try:
            issuess = json.loads(r.content)
            for i in issuess:
                issue = i['body']
                if ('exception' in issue) or ('Exception' in issue) or ('EXCEPTION' in issue): 
                    issues.append(issue)
                    
                    open('issues/exceptions/' + str(count), 'w').write(issue)
                    count +=1
                    print(count)
                else:
                    skip +=1
                    print('unrelated:- '+str(skip))
                    open('issues/' + str(skip), 'w').write(issue)
        except:
            print('request failed')

### Reading from Downloaded Files

In [3]:
import glob
root_dir = 'issues/exceptions/'
issues = []
for filepath in glob.iglob(root_dir + '**/**', recursive=True):
     if os.path.isfile(filepath):
            with open(filepath, 'r') as f:
                issues.append(f.read())

#### Filtering Java Issues

In [116]:
java_issues = []
for issue in issues:
    if 'java.lang' in issue:
        java_issues.append(issue)
len(java_issues)

10025

### Getting Errors and Exceptions

In [117]:
import re
err = re.compile('java.lang.*')
excp = re.compile('java.lang.*')
Errors = []
Exceptions = []
miss_err = 0
miss_exception = 0
for  issue in java_issues:
    for i in issue.split(' '):
        if 'java.lang.' in i and ('Exception' in i or 'Error' in i):
            try:
                if 'Error' in i:
                    error = err.findall(issue)[0]
                    ind = error.find('Error')+5
                    Errors.append(error[:ind])
                    print(error[:ind])
            except:
                miss_err +=1
            try:
                if 'Exception' in i:
                    exception = excp.findall(issue)[0]
                    ind = exception.find('Exception')+9
                    Exceptions.append(exception[:ind])
                    print(exception[:ind])
            except:
                miss_exception +=1
        break

Total_Errors = Errors + Exceptions
len(Total_Errors)

java.lang.IllegalArgumentException
java.lang.RuntimeException
java.lang.IllegalStateException
java.lang.NullPointerException
java.lang.NullPointerException
java.lang.ClassCastException
java.lang.RuntimeException
java.lang.StringIndexOutOfBoundsException
java.lang.ClassCastException
java.lang.ArrayIndexOutOfBoundsException
java.lang.NullPointerException
java.lang.NullPointerException
java.lang.ArrayIndexOutOfBoundsException
java.lang.IllegalStateException
java.lang.NoClassDefFoundError
java.lang.IllegalMonitorStateException
java.lang.IllegalStateException
java.lang.NullPointerException
java.lang.NullPointerException
java.lang.IndexOutOfBoundsException
java.lang.RuntimeException
java.lang.IllegalStateException
java.lang.NullPointerException
java.lang.NullPointerException
java.lang.ClassCastException
java.lang.NullPointerException
java.lang.module.ResolutionException
java.lang.ClassCastException
java.lang.RuntimeException
java.lang.RuntimeException
java.lang.IllegalArgumentException
java.

493

### Saving into Dictionary

In [122]:
values, counts = np.unique(Total_Errors, return_counts=True)
dictionary = dict(zip(values, counts))
dictionary

{'java': 2,
 'java.lan': 2,
 'java.lang.AbstractMethodError': 1,
 'java.lang.ArithmeticException': 2,
 'java.lang.ArrayIndexOutOfBoundsException': 25,
 'java.lang.ArrayStoreException': 1,
 'java.lang.ClassCastException': 27,
 'java.lang.ClassNotFoundException': 11,
 'java.lang.Error': 1,
 'java.lang.Exception': 8,
 'java.lang.ExceptionInInitializerError': 5,
 'java.lang.IllegalAccessException': 2,
 'java.lang.IllegalArgumentException': 58,
 'java.lang.IllegalMonitorStateException': 1,
 'java.lang.IllegalStateException': 66,
 'java.lang.IndexOutOfBoundsException': 12,
 'java.lang.NegativeArraySizeException': 1,
 'java.lang.NoClassDefFoundError': 8,
 'java.lang.NoSuchFieldException': 2,
 'java.lang.NoSuchMethodException': 2,
 'java.lang.NoclassDeFoundError': 1,
 'java.lang.NullPointerException': 142,
 'java.lang.NumberFormatException': 4,
 'java.lang.OutOfMemoryError': 2,
 'java.lang.RuntimeException': 80,
 'java.lang.RuntimeException:Error': 1,
 'java.lang.SecurityException': 8,
 'java.

In [123]:
### Dropping Garbage
dictionary.pop('java')
dictionary.pop('java.lan')

2

### Writing Dictionary into File

In [126]:
with open('dictionary.txt','w+') as file:
    file.write(str(dictionary))